In [1]:
import cv2               as cv
import urllib
import numpy             as np
import pandas            as pd
import seaborn           as sns
import random
import os
import glob
import shutil
from imutils        import paths
import matplotlib.pyplot as plt
from sklearn.utils  import shuffle
from urllib.request import urlopen
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models              import Sequential
from tensorflow.keras.preprocessing       import image
from tensorflow.keras.utils               import to_categorical
from tensorflow.keras.callbacks           import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers              import Conv2D,Flatten,MaxPooling2D,Dense,Dropout,SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img,array_to_img

## 1-) Data Preparation

Data Preprocessing + Data Augmentation

In [2]:
folder_path = "Dataset\Dataset_nonsplit"

target_size = (240,240)

labels = {"Brain Tumor":0,
          "Healthy"    :1}

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
#
#Folders
original_dataset_folder = "Dataset\Dataset_nonsplit"
base_folder = 'Dataset/Dataset_Split'
os.makedirs(base_folder, exist_ok=True)
#Create Folders
train_folder = os.path.join(base_folder, 'train')
os.makedirs(train_folder, exist_ok=True)
test_folder = os.path.join(base_folder, 'test')
os.makedirs(test_folder, exist_ok=True)
#Create Class folders
classes = os.listdir(original_dataset_folder)
for class_ in classes:
    os.makedirs(os.path.join(train_folder, class_), exist_ok=True)
    os.makedirs(os.path.join(test_folder, class_), exist_ok=True)
#Copy images
for class_ in classes:
    src_folder = os.path.join(original_dataset_folder, class_)
    all_filenames = os.listdir(src_folder)
    # 80% train - 20% test
    train_filenames, test_filenames = train_test_split(all_filenames, test_size=0.2, random_state=42)
    # Copy
    for filename in train_filenames:
        shutil.copy(os.path.join(src_folder, filename), os.path.join(train_folder, class_, filename))
    for filename in test_filenames:
        shutil.copy(os.path.join(src_folder, filename), os.path.join(test_folder, class_, filename))

In [3]:
#Data augmentation is only for train set.
train_datagen = ImageDataGenerator(rescale           =1.0/255, # Normalization
                                   zoom_range        =0.2,     # Random zoom
                                   width_shift_range =0.2,     # Random shift
                                   height_shift_range=0.2,     # Random shift
                                   validation_split  =0.125)   # 10% for validation split
#No augmentation for test set.
test_datagen  = ImageDataGenerator(rescale = 1.0/255)
